## Read data

In [3]:
import pandas as pd
feat = pd.read_csv("/home/aistudio/data/data22449/date_feat.csv")
label = pd.read_csv("/home/aistudio/data/data22450/label.csv")
customer = pd.read_csv("work/customer.csv")

In [4]:
#customer

In [5]:
nameli = label.columns
nameli

Index(['2012.11', '2012.12', '2013.1', '2013.2', '2013.3', '2013.4', '2013.5',
       '2013.6', '2013.7', '2013.8'],
      dtype='object')

In [6]:
import numpy as np

In [7]:
features = feat.values
#features

In [29]:
f1 = features[:,0:3]
f2 = features[:,3:7]
f3 = features[:,7:]

In [30]:
feat1 = [[x] for x in np.sum(f1,axis= 1)]
feat2 = [[x] for x in np.sum(f2,axis= 1)]
feat3 = [[x] for x in np.sum(f3,axis= 1)]

### Get train set

In [ ]:
## train_set:== 3,4,5,6-->7
##              4,5,6,7-->8
##              5,6,7,8-->9

In [35]:
Xa = features[:,3:7]
Xa = np.hstack((Xa,customer,feat1))
ya = label[nameli[7]].values

In [36]:
Xb = features[:,4:8]
Xb = np.hstack((Xb,customer,feat1))
yb = label[nameli[8]].values

In [37]:
Xc = features[:,5:9]
Xc = np.hstack((Xc,customer,feat1))
yc = label[nameli[9]].values

In [50]:
X = np.vstack((Xa,Xb,Xc))
y = np.hstack((ya,yb,yc))

In [47]:
X.shape

(4757958, 8)

In [51]:
y.shape

(4757958,)

In [40]:
X1 = features[:,6:10]
X1 = np.hstack((X1,customer,feat1))
y1 = label[nameli[9]].values

In [44]:
X.shape

(4757958, 8)

## focal LightGbm

In [45]:
def sigmoid(x): 
    return 1./(1. +  np.exp(-x))

def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / (np.sum(exp_x, axis=1, keepdims=True) + 1e-6)

In [16]:
def focal_loss_lgb(y_pred, dtrain, alpha, gamma, num_class):
    """
    Focal Loss for lightgbm

    Parameters:
    -----------
    y_pred: numpy.ndarray
        array with the predictions
    dtrain: lightgbm.Dataset
    alpha, gamma: float
        See original paper https://arxiv.org/pdf/1708.02002.pdf
    num_class: int
        number of classes
    """
    a,g = alpha, gamma
    y_true = dtrain.label
    # N observations x num_class arrays
    y_true = np.eye(num_class)[y_true.astype('int')]
    y_pred = y_pred.reshape(-1,num_class, order='F')
    # alpha and gamma multiplicative factors with BCEWithLogitsLoss
    def fl(x,t):
        p = 1/(1+np.exp(-x))
        return -( a*t + (1-a)*(1-t) ) * (( 1 - ( t*p + (1-t)*(1-p)) )**g) * ( t*np.log(p)+(1-t)*np.log(1-p) )
    partial_fl = lambda x: fl(x, y_true)
    grad = derivative(partial_fl, y_pred, n=1, dx=1e-6)
    hess = derivative(partial_fl, y_pred, n=2, dx=1e-6)
    # flatten in column-major (Fortran-style) order
    return grad.flatten('F'), hess.flatten('F')

In [17]:
def focal_loss_lgb_eval_error(y_pred, dtrain, alpha, gamma, num_class):
    """
    Focal Loss for lightgbm

    Parameters:
    -----------
    y_pred: numpy.ndarray
        array with the predictions
    dtrain: lightgbm.Dataset
    alpha, gamma: float
        See original paper https://arxiv.org/pdf/1708.02002.pdf
    num_class: int
        number of classes
    """
    a,g = alpha, gamma
    y_true = dtrain.label
    y_true = np.eye(num_class)[y_true.astype('int')]
    y_pred = y_pred.reshape(-1, num_class, order='F')
    p = 1/(1+np.exp(-y_pred))
    loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
    # a variant can be np.sum(loss)/num_class
    return 'focal_loss', np.mean(loss), False

In [19]:
import numpy as np
import lightgbm as lgb

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import  accuracy_score
from scipy.misc import derivative

# very inadequate dataset as is perfectly balanced, but just to illustrate
#digits = datasets.load_digits()
#X = digits.data
#y = digits.target


In [52]:
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.25, random_state=1)
lgbtrain = lgb.Dataset(X_tr, y_tr, free_raw_data=True)
lgbeval = lgb.Dataset(X_val, y_val)

In [74]:
focal_loss = lambda x,y: focal_loss_lgb(x, y, 0.1, 2., 2)
eval_error = lambda x,y: focal_loss_lgb_eval_error(x, y, 0.1, 2., 2)
params  = {'learning_rate':0.1, 'num_boost_round':20, 'num_class':2}#,"reg_alpha":0.5, "reg_lambda":0.8}
# model = lgb.train(params, lgbtrain, fobj=focal_loss)
model = lgb.train(params, lgbtrain, valid_sets=[lgbeval], fobj=focal_loss, feval=eval_error)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's focal_loss: 0.0771087
[2]	valid_0's focal_loss: 0.0689107
[3]	valid_0's focal_loss: 0.0618379
[4]	valid_0's focal_loss: 0.055716
[5]	valid_0's focal_loss: 0.0504037
[6]	valid_0's focal_loss: 0.0457813
[7]	valid_0's focal_loss: 0.0417533
[8]	valid_0's focal_loss: 0.0382355
[9]	valid_0's focal_loss: 0.0351591
[10]	valid_0's focal_loss: 0.0324645
[11]	valid_0's focal_loss: 0.0301029
[12]	valid_0's focal_loss: 0.0280296
[13]	valid_0's focal_loss: 0.0262083
[14]	valid_0's focal_loss: 0.0246058
[15]	valid_0's focal_loss: 0.0231952
[16]	valid_0's focal_loss: 0.0219524
[17]	valid_0's focal_loss: 0.0208573
[18]	valid_0's focal_loss: 0.019891
[19]	valid_0's focal_loss: 0.0190383
[20]	valid_0's focal_loss: 0.0182851


In [75]:
w1= 4.5

def eval_metric(l,y):
    p = 1.0/(1.0+np.exp(-y))
    cross =-sum( w1*l*np.log(p)+(1-l)*np.log(1-p))/1585986
    print("cross+++",cross)
    score = 100 - (cross-0.6)*100
    return score

In [76]:
ans = model.predict(X1)
ans

array([[ 0.93299034, -1.13075003],
       [ 0.38294125, -1.10210389],
       [-0.46514325, -0.8270531 ],
       ...,
       [ 0.40197925, -1.12032918],
       [ 0.66328489, -1.12032918],
       [-0.03478062, -1.04240361]])

In [77]:
y_pred = sigmoid(ans)[:,1]
y_pred

array([0.24402271, 0.2493459 , 0.30426854, ..., 0.24595023, 0.24595023,
       0.26068648])

In [69]:
eval_metric(y1,y_pred)

cross+++ 0.9756889730609354


62.43110269390646

In [58]:
!rm -rf submit.sh
!wget -O submit.sh http://ai-studio-static.bj.bcebos.com/script/submit.sh

--2020-02-29 01:48:07--  http://ai-studio-static.bj.bcebos.com/script/submit.sh
Resolving ai-studio-static.bj.bcebos.com (ai-studio-static.bj.bcebos.com)... 100.64.253.38, 100.64.253.37
Connecting to ai-studio-static.bj.bcebos.com (ai-studio-static.bj.bcebos.com)|100.64.253.38|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 340 [text/x-sh]
Saving to: ‘submit.sh’

submit.sh           100%[===================>]     340  --.-KB/s    in 0s      

2020-02-29 01:48:07 (116 MB/s) - ‘submit.sh’ saved [340/340]



In [59]:
subm = pd.read_csv("work/submission.csv")

In [64]:
subm.head(5)

,customer_id,result
0,1000000,0.052968
1,1000014,0.110468
2,1000034,0.256183
3,1000046,0.071670
4,1000048,0.051451


In [78]:
subm['result'] = y_pred

In [79]:
subm.to_csv("subm.csv" ,index = False)

In [80]:
!sh submit.sh "subm.csv" 02096e5f1617405ebcaa63308c18cb66

--2020-02-29 02:19:47--  http://ai-studio-static.bj.bcebos.com/script/bce-python-sdk.zip
Resolving ai-studio-static.bj.bcebos.com (ai-studio-static.bj.bcebos.com)... 100.64.253.37, 100.64.253.38
Connecting to ai-studio-static.bj.bcebos.com (ai-studio-static.bj.bcebos.com)|100.64.253.37|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80914 (79K) [application/zip]
Saving to: ‘bce-python-sdk.zip’

bce-python-sdk.zip  100%[===================>]  79.02K  --.-KB/s    in 0.004s  

2020-02-29 02:19:47 (17.4 MB/s) - ‘bce-python-sdk.zip’ saved [80914/80914]

Archive:  bce-python-sdk.zip
   creating: bce-python-sdk/
   creating: bce-python-sdk/baidubce/
  inflating: bce-python-sdk/baidubce/.DS_Store  
   creating: bce-python-sdk/baidubce/.idea/
  inflating: bce-python-sdk/baidubce/.idea/baidubce.iml  
   creating: bce-python-sdk/baidubce/.idea/inspectionProfiles/
   creating: bce-python-sdk/baidubce/.idea/markdown-navigator/
  inflating: bce-python-sdk/baidubce/.idea/mark